Primero cargaremos todo las librerias que usaremos y tambien el archivo.csv de canciones

In [2]:
import pandas as pd
import numpy as np
import librosa
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

songs= pd.read_csv('../spotify_songs_en.csv')
songs.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28.0,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1.0,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512.0,en
1,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0.0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0.0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467.0,en
2,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41.0,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0.0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067.0,en
3,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65.0,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1.0,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160.0,en
4,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,"Hold your breath, don't look down, keep trying...",70.0,3GNzXsFbzdwM0WKCZtgeNP,Chapters,2019-04-26,urban contemporary,4WiB26kw0INKwbzfb5M6Tv,...,-6.157,1.0,0.0550,0.28000,0.00000,0.0975,0.305,147.764,224720.0,en


Cargamos el csv de vector caracteristico de las canciones

In [3]:
features = pd.read_csv('../new_features.csv')
features.head()

,track_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,0zEKOcdfzNkvW1OFnazAQO,-0.098841,-0.116778,-0.005992,-0.076890,0.167060,-0.153227,-0.127395,0.062021,-0.143093,...,-0.134901,-0.126881,-0.211531,-0.137909,0.010951,0.330660,0.048009,0.161574,0.138966,0.203755
1,0ZfM5XfJTtFPhOxAERRnNY,-0.329168,-0.208267,-0.242830,0.158825,-0.002144,0.104842,0.214938,0.311463,0.014895,...,-0.117848,0.411027,0.117565,0.494875,0.159988,-0.554037,-0.252416,-0.382986,-0.372454,-0.487503
2,0zG6VemmYlMhAK5MNvBeP7,0.505574,0.732437,-0.009727,-0.097279,-0.436935,-0.585198,-0.322022,-0.157966,-0.272551,...,-0.062521,-0.071936,-0.313199,0.065619,0.130820,-0.027184,0.255665,0.166922,0.731590,0.160565
3,0ZGLuduCPjgWY1n85ykgMe,-0.155610,0.135038,-0.230663,-0.338267,-0.108828,0.285991,0.326603,0.131136,-0.333797,...,-0.213955,-0.266517,-0.115268,-0.004639,-0.237066,-0.114990,-0.064895,0.224829,-0.642008,0.316891
4,0ZgTez1pojM5aCxsV0hrC9,0.157820,-0.082984,-0.005262,-0.172845,-0.112333,-0.223888,-0.055531,0.009561,-0.095176,...,-0.165788,-0.390585,0.204284,0.097674,0.198702,0.039575,-0.120027,-0.157962,-0.197737,0.215317


Verficamos que tenemos lo necesario par auna busqueda textual con pandas solo extraemos las columnas "track_name", "track_artist", "lyrics","track_id". Esto no es necesario, pero se implemento con KNN.

In [4]:
essential = songs[["track_name", "track_artist", "lyrics","track_id"]]
essential.head()

,track_name,track_artist,lyrics,track_id
0,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",004s3t0ONYlzxII9PLgU6z
1,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",00chLpzhgVjxs1zKC9UScL
2,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,00cqd6ZsSkLZqGMlQCR0Zo
3,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,00emjlCv9azBN0fzuuyLqy
4,Soldier,James TW,"Hold your breath, don't look down, keep trying...",00f9VGHfQhAHMCQ2bSjg3D


Limpieza de datos nulos y duplicados

In [5]:
essential = essential.dropna()
essential = essential.drop_duplicates()
essential.shape
essential.head()

,track_name,track_artist,lyrics,track_id
0,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",004s3t0ONYlzxII9PLgU6z
1,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",00chLpzhgVjxs1zKC9UScL
2,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,00cqd6ZsSkLZqGMlQCR0Zo
3,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,00emjlCv9azBN0fzuuyLqy
4,Soldier,James TW,"Hold your breath, don't look down, keep trying...",00f9VGHfQhAHMCQ2bSjg3D


Obtenemos el tfidvectorizer de nuestra dataframe

In [6]:
vectorizer = TfidfVectorizer()
vectorizer
X = vectorizer.fit_transform(essential["lyrics"])
X

<15408x47584 sparse matrix of type '<class 'numpy.float64'>'
	with 1746102 stored elements in Compressed Sparse Row format>

KNN secuencial de busqueda textual implementada

In [7]:
def KNN_secuencial(query, k):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X,query_vec).reshape((-1,))
    top_k_indices = results.argsort()[-k:][::-1]
    top_k_songs = essential.iloc[top_k_indices][["track_name", "track_artist"]]
    return top_k_songs
query = "I'm a rebel just for kicks, now"
k=5
print(KNN_secuencial(query, k))




                                 track_name       track_artist
14497                         Feel It Still  Portugal. The Man
13233                         Feel It Still  Portugal. The Man
1670    It's All on U - T-Mass & LZRD Remix           ILLENIUM
7309   It's All on U (feat. Liam O'Donnell)           ILLENIUM
10528           Rebel Rebel - 1997 Remaster        David Bowie


Tambien se implemento una variacion con heap

In [8]:
import heapq
def KNN_secuencial_heap(query, k):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X,query_vec).reshape((-1,))
    top_k_indices = heapq.nlargest(k, range(len(results)), results.take)
    top_k_songs = essential.iloc[top_k_indices][["track_name", "track_artist"]]
    return top_k_songs

print(KNN_secuencial_heap(query, k))

                                 track_name       track_artist
13233                         Feel It Still  Portugal. The Man
14497                         Feel It Still  Portugal. The Man
1670    It's All on U - T-Mass & LZRD Remix           ILLENIUM
7309   It's All on U (feat. Liam O'Donnell)           ILLENIUM
4581            Rebel Rebel - 2016 Remaster        David Bowie


Se implemento una busqueda textual con knn_range_search, no es necesario

In [9]:
radius = 0.2
def KNN_range_Search(query, radius):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X, query_vec).reshape((-1,))
    indices = np.where(results >= radius)[0]
    songs = essential.iloc[indices][["track_name", "track_artist"]]
    return songs

print(KNN_range_Search(query, radius))

                                 track_name       track_artist
1670    It's All on U - T-Mass & LZRD Remix           ILLENIUM
4581            Rebel Rebel - 2016 Remaster        David Bowie
7309   It's All on U (feat. Liam O'Donnell)           ILLENIUM
8332                              Pumped Up          Klingande
9609                                McQueen          TrifeDrew
10528           Rebel Rebel - 1997 Remaster        David Bowie
12042           Rebel Rebel - 1999 Remaster        David Bowie
12907                             Pumped Up          Klingande
13178           Rebel Rebel - 2014 Remaster        David Bowie
13226              Rebel Rebel - Remastered        David Bowie
13233                         Feel It Still  Portugal. The Man
14216                         Teenage Kicks     The Undertones
14497                         Feel It Still  Portugal. The Man
15308                       Pumped Up Kicks  Foster The People


In [10]:
first_value = songs.loc[0]
print(first_value)

track_id                                               004s3t0ONYlzxII9PLgU6z
track_name                                                       I Feel Alive
track_artist                                                    Steady Rollin
lyrics                      The trees, are singing in the wind The sky blu...
track_popularity                                                         28.0
track_album_id                                         3z04Lb9Dsilqw68SHt6jLB
track_album_name                                                  Love & Loss
track_album_release_date                                           2017-11-21
playlist_name                                               Hard Rock Workout
playlist_id                                            3YouF0u7waJnolytf9JCXf
playlist_genre                                                           rock
playlist_subgenre                                                   hard rock
danceability                                                    

Esta parte es lo que se pidio implementar, extraemos las mismas columnas importantes "track_name", "track_artist", "lyrics","track_id" para verificar si resulto con existe vemos las primeras filas con .head() de nuestra dataframe.

In [11]:
essential_2 = songs[['track_id', 'track_name', 'track_artist', 'lyrics']]
essential_2.head()

,track_id,track_name,track_artist,lyrics
0,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu..."
1,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U..."
2,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...
3,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...
4,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,"Hold your breath, don't look down, keep trying..."


Limpieza de datos:

- **Eliminamos Datos nulos**

- **Eliminamos Datos duplicados**

Verificamos la cantidad de elementos se redujo o se mantuvo

In [12]:
essential_2 = essential_2.dropna()
essential_2 = essential_2.drop_duplicates()
essential_2.shape


(15408, 4)

Hacemos el merge entre nuestra dataframe extraida essential2 y nuestro csv con el vector caracteristico cargado anteriormente.


In [13]:
combined_data = pd.merge(essential_2, features, on='track_id')
combined_data

,track_id,track_name,track_artist,lyrics,f1,f2,f3,f4,f5,f6,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",0.045167,0.125351,0.270290,-0.034200,-0.112064,0.084561,...,-0.443242,0.084999,-0.412624,0.400742,0.204244,-0.333002,0.518897,0.039950,0.185921,-0.557007
1,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0.364316,0.343706,-0.195609,0.008802,0.452232,-0.079514,...,0.190457,0.202202,0.133856,-0.452725,-0.338745,-0.050033,0.028329,0.140822,0.259863,-1.065271
2,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,0.040475,-0.138258,-0.016978,0.032210,0.061145,-0.143123,...,0.121772,-0.302128,-0.131246,-0.228789,-0.362044,-0.027669,0.188981,0.245499,-0.523468,0.374002
3,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,-0.330823,0.310332,-0.003732,-0.220356,-0.297301,0.333718,...,-0.667551,0.135213,0.413494,-0.125062,-0.056369,0.409041,-0.067511,0.096366,-0.513577,-1.521303
4,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,"Hold your breath, don't look down, keep trying...",-0.530786,-0.656363,-0.583888,-0.444300,-0.298558,0.384282,...,-0.173826,-0.041576,0.152726,-0.372270,-0.232274,-0.037420,0.224199,0.278720,0.252481,1.087486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13193,7K4qs6xArR9izybon4i4gW,Somebody,Erlandsson,"I've been high, I've been low Chasing rivers a...",-0.202367,-0.473218,-0.447030,-0.149925,-0.152570,-0.259613,...,0.174262,0.318154,0.152563,0.321650,0.298896,0.101073,0.200171,-0.079855,0.019494,0.178448
13194,7K5dzhGda2vRTaAWYI3hrb,Just Like Fire (From the Original Motion Pictu...,P!nk,I know that I am runnin' out of time I want it...,-0.451623,-0.234071,-0.159290,-0.161218,-0.095926,-0.051664,...,-0.119346,-0.139924,-0.272772,-0.386242,0.218397,0.275528,0.536367,0.256531,0.399271,-0.691504
13195,7k5lLjCSnyUB4S1LRvD3I4,Here We Go Again! - Extended,Portrait,"(Chuck D: Here we go again) Well, I'm out to h...",0.656685,0.437153,0.526300,0.198158,0.008960,-0.278656,...,-0.006828,-0.205763,0.011333,0.094089,0.197834,0.163011,0.098527,-0.196720,-0.258613,0.252062
13196,7k6IzwMGpxnRghE7YosnXT,Me & U,Cassie,"Uh, Next Selection You've been waiting so long...",0.884435,0.957433,0.266550,-0.403613,-0.411181,0.208359,...,0.028934,0.065975,0.331138,0.341994,0.015794,-0.213025,-0.156314,-0.068632,-0.226346,-0.478887


Limipieza de datos luego del merge:

- **Eliminamos Datos nulos**

- **Eliminamos Datos duplicados**



In [ ]:
normalized_combined_data = combined_data.dropna() #Elimino la tabla con valores nulos
normalized_combined_data = combined_data.drop_duplicates() #Elimino los duplicados

normalized_combined_data.shape

(13198, 364)

In [15]:

# Seleccionar solo las columnas numéricas
features_to_normalize = combined_data.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
features_to_normalize.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,0.045167,0.125351,0.270290,-0.034200,-0.112064,0.084561,0.030843,0.466074,0.163689,0.139688,...,-0.443242,0.084999,-0.412624,0.400742,0.204244,-0.333002,0.518897,0.039950,0.185921,-0.557007
1,0.364316,0.343706,-0.195609,0.008802,0.452232,-0.079514,-0.167727,-0.010781,-0.022688,-0.129539,...,0.190457,0.202202,0.133856,-0.452725,-0.338745,-0.050033,0.028329,0.140822,0.259863,-1.065271
2,0.040475,-0.138258,-0.016978,0.032210,0.061145,-0.143123,0.369927,-0.110431,0.088467,-0.037289,...,0.121772,-0.302128,-0.131246,-0.228789,-0.362044,-0.027669,0.188981,0.245499,-0.523468,0.374002
3,-0.330823,0.310332,-0.003732,-0.220356,-0.297301,0.333718,0.013620,0.194298,0.541525,-0.228518,...,-0.667551,0.135213,0.413494,-0.125062,-0.056369,0.409041,-0.067511,0.096366,-0.513577,-1.521303
4,-0.530786,-0.656363,-0.583888,-0.444300,-0.298558,0.384282,0.191833,0.449212,0.248942,-0.009043,...,-0.173826,-0.041576,0.152726,-0.372270,-0.232274,-0.037420,0.224199,0.278720,0.252481,1.087486


Limpieza de datos a esta nueva dataframe:

- **Verificar si hay valores NaN en el DataFrame**

- **Mostrar las filas con al menos un valor NaN**

- **Mostrar el número total de filas con al menos un valor NaN**

In [19]:
nan_check = normalized_combined_data.isna().any(axis=1)

rows_with_nan = normalized_combined_data[nan_check]
print(f"Filas con al menos un valor NaN:\n{rows_with_nan}")

total_rows_with_nan = rows_with_nan.shape[0]
print(f"Número total de filas con al menos un valor NaN: {total_rows_with_nan}")


Filas con al menos un valor NaN:
Empty DataFrame
Columns: [track_id, track_name, track_artist, lyrics, f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f14, f15, f16, f17, f18, f19, f20, f21, f22, f23, f24, f25, f26, f27, f28, f29, f30, f31, f32, f33, f34, f35, f36, f37, f38, f39, f40, f41, f42, f43, f44, f45, f46, f47, f48, f49, f50, f51, f52, f53, f54, f55, f56, f57, f58, f59, f60, f61, f62, f63, f64, f65, f66, f67, f68, f69, f70, f71, f72, f73, f74, f75, f76, f77, f78, f79, f80, f81, f82, f83, f84, f85, f86, f87, f88, f89, f90, f91, f92, f93, f94, f95, f96, ...]
Index: []

[0 rows x 364 columns]
Número total de filas con al menos un valor NaN: 0


Verficamos nuestra dataframe limpia.

In [20]:
normalized_combined_data

,track_id,track_name,track_artist,lyrics,f1,f2,f3,f4,f5,f6,...,f351,f352,f353,f354,f355,f356,f357,f358,f359,f360
0,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",0.298782,0.487237,0.913686,-0.021343,-0.339007,0.335685,...,-1.464961,0.304769,-1.397211,1.291972,0.580000,-1.339713,1.597153,0.006066,0.439929,-0.736883
1,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0.926424,1.027359,-0.447941,0.120130,1.609441,-0.260902,...,0.634925,0.696336,0.434605,-1.619941,-1.332335,-0.345428,-0.055814,0.315409,0.621152,-1.559313
2,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,0.289554,-0.164826,0.074122,0.197138,0.259063,-0.492190,...,0.407324,-0.988600,-0.454025,-0.855902,-1.414392,-0.266849,0.485503,0.636420,-1.298719,0.769596
3,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,-0.440645,0.944805,0.112835,-0.633781,-0.978609,1.241643,...,-2.208252,0.472530,1.371962,-0.501998,-0.337845,1.267639,-0.378748,0.179077,-1.274477,-2.297225
4,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,"Hold your breath, don't look down, keep trying...",-0.833895,-1.446406,-1.582717,-1.370534,-0.982949,1.425497,...,-0.572196,-0.118113,0.497858,-1.345438,-0.957359,-0.301109,0.604171,0.738297,0.603061,1.924096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13193,7K4qs6xArR9izybon4i4gW,Somebody,Erlandsson,"I've been high, I've been low Chasing rivers a...",-0.188022,-0.993380,-1.182740,-0.402069,-0.478869,-0.915759,...,0.581258,1.083723,0.497311,1.022123,0.913352,0.185520,0.523208,-0.361337,0.032031,0.453167
13194,7K5dzhGda2vRTaAWYI3hrb,Just Like Fire (From the Original Motion Pictu...,P!nk,I know that I am runnin' out of time I want it...,-0.678213,-0.401826,-0.341795,-0.439222,-0.283285,-0.159638,...,-0.391666,-0.446688,-0.928424,-1.393111,0.629843,0.798510,1.656017,0.670252,0.962828,-0.954515
13195,7k5lLjCSnyUB4S1LRvD3I4,Here We Go Again! - Extended,Portrait,"(Chuck D: Here we go again) Well, I'm out to h...",1.501402,1.258509,1.661897,0.743092,0.078876,-0.985001,...,-0.018817,-0.666652,0.023904,0.245715,0.557424,0.403153,0.180717,-0.719724,-0.649583,0.572283
13196,7k6IzwMGpxnRghE7YosnXT,Me & U,Cassie,"Uh, Next Selection You've been waiting so long...",1.949299,2.545469,0.902757,-1.236678,-1.371823,0.785829,...,0.099688,0.241210,1.095899,1.091533,-0.083698,-0.918142,-0.677969,-0.326921,-0.570500,-0.610477


Esta parte es necesaria para la query de modo que crearemos un vector caracteristico para esta consulta, necesariamente la misma cantidad de caracteristicas que de las canciones.

## Funcion de extraccion de caracteristicas del audio

In [ ]:
scaler = preprocessing.StandardScaler()
def extract_features(file_path):
    # Cargar el archivo de audio
    y, sr = librosa.load(file_path)

    # Extraer características
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    
    #Estandarizamos
    chroma_stft = scaler.fit_transform(chroma_stft.T)
    chroma_stft = chroma_stft.T
    # Dividir en segmentos uniformes
    num_segments = 30
    segment_len = chroma_stft.shape[1] // num_segments

    # Tomar la media de cada segmento
    chroma_stft_uniform = np.mean(chroma_stft[:, :num_segments * segment_len].reshape(12, -1, segment_len), axis=2)
    # Concatenar todas las características en un solo vector
    features = chroma_stft_uniform.flatten()
    
    return features

### Consulta 1

In [ ]:
audio_path="../feel.mp3"
scaled_audio_features=pd.DataFrame(extract_features(audio_path)).transpose()
scaled_audio_features

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,0.373056,0.107478,0.184831,-0.162873,-0.117238,0.217079,0.075112,0.650628,0.10612,0.021485,...,-0.410744,0.101319,-0.287646,0.274538,0.236566,-0.270106,0.438478,0.154848,0.114364,-0.638817


### Consulta 2

In [ ]:
audio_path="../query2.wav"
scaled_audio_features2=pd.DataFrame(extract_features(audio_path)).transpose()
scaled_audio_features2

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,1.324466,0.022579,0.309826,0.806057,0.971554,0.526042,0.09857,-0.865484,1.279644,1.091261,...,-0.915616,-1.080968,-1.216198,0.449409,0.800919,0.639257,-0.287134,-0.891652,-1.021489,-0.421016


### Consulta 3

In [ ]:
audio_path="../cinderella.wav"
scaled_audio_features3=pd.DataFrame(extract_features(audio_path)).transpose()
scaled_audio_features3

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,-1.039753,0.034764,0.102336,-0.440019,-0.426903,0.748025,-0.372351,-0.257574,-0.365496,0.970163,...,0.287135,0.209575,0.009633,0.313954,0.362166,-0.040241,0.262616,-0.077819,0.112796,-0.562266


## MODELO KNN

EL modelo KNN search se implementa con ayudade sklearn que nos proporciona la facilidad con los modelos de un arbol KD conseguir los vecinos mas cercanos

### Sin cola de prioridad

In [ ]:
from sklearn.neighbors import NearestNeighbors

def knn_search(query_audio_features, reference_features, k=5, metric='euclidean'):
    # Inicializar el modelo KNN
    knn_model = NearestNeighbors(n_neighbors=k, metric=metric)

    # Extraer las características numéricas para el modelo KNN, 4 primeras columnas del data frame
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
    knn_model.fit(reference_features_numeric)

    # Convertir query_audio_features a un array NumPy
    query_features_numeric = query_audio_features.to_numpy()

    # Utilizar el modelo para encontrar los k vecinos más cercanos para la query del audio
    distances, indices = knn_model.kneighbors(query_features_numeric)

    # Obtener las k canciones más cercanas utilizando los índices
    nearest_songs = reference_features.iloc[indices[0], :4]  # Obtener solo las primeras 4 columnas

    return nearest_songs


### Con cola de prioridad

In [ ]:
from queue import PriorityQueue
from sklearn.neighbors import NearestNeighbors

def knn_search_priority_queue(query_audio_features, reference_features, k=5, metric='euclidean'):
    # Initialize the KNN model
    knn_model = NearestNeighbors(n_neighbors=k, metric=metric)

    # Extract the numeric features for the KNN model, excluding the first 4 columns
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
    knn_model.fit(reference_features_numeric)

    # Convert query_audio_features to a NumPy array
    query_features_numeric = query_audio_features.to_numpy()

    # Use the model to find the k nearest neighbors for the audio query
    distances, indices = knn_model.kneighbors(query_features_numeric)

    # Create a priority queue to store the k nearest neighbors
    pq = PriorityQueue()

    # Iterate over the indices and distances to add the neighbors to the priority queue
    for i in range(len(indices[0])):
        index = indices[0][i]
        distance = distances[0][i]
        pq.put((distance, index))

    # Get the k nearest neighbors from the priority queue
    nearest_songs = []
    while not pq.empty():
        distance, index = pq.get()
        nearest_songs.append(reference_features.iloc[index, :4])  # Get only the first 4 columns

    return nearest_songs


## Visto en clase

In [ ]:
def knn_search_simple(Q, collection, k):
    result = []

    for _, row in collection.iterrows():
        C = row.iloc[4:].to_numpy()  # Obtener el vector característico desde la quinta columna hasta el final

        # Calcular la distancia para cada fila de la matriz de consulta Q
        distances = [distance(q, C) for q in Q]

        # Obtener el índice del elemento más cercano para cada fila de Q
        indices = np.argsort(distances)

        # Agregar resultados a la lista
        for i in range(len(Q)):
            result.append({
                'track_id': row['track_id'],
                'track_name': row['track_name'],
                'track_artist': row['track_artist'],
                'distance': distances[i],
                'query_row': i
            })

    result.sort(key=lambda x: x['distance'])  # Ordena por distancia en orden ascendente
    return result[:k+1]  # Devuelve los k vecinos más cercanos




def distance(vector1, vector2):
    return np.linalg.norm(vector1 - vector2)

### Con Heap

In [ ]:
import heapq

def knn_search_simple_priority_queue(Q, collection, k):
    result = []

    for _, row in collection.iterrows():
        C = row.iloc[4:].to_numpy()  # Obtener el vector característico desde la quinta columna hasta el final

        # Calcular la distancia para cada fila de la matriz de consulta Q
        distances = [distance(q, C) for q in Q]

        # Obtener el índice del elemento más cercano para cada fila de Q utilizando una cola de prioridad
        closest_indices = heapq.nsmallest(k, range(len(distances)), key=distances.__getitem__)

        # Agregar resultados a la lista
        for i in closest_indices:
            result.append({
                'track_id': row['track_id'],
                'track_name': row['track_name'],
                'track_artist': row['track_artist'],
                'distance': distances[i],
                'query_row': i
            })

    result.sort(key=lambda x: x['distance'])  # Ordena por distancia en orden ascendente
    return result[:k]


## MODELO KNN - arboles KD

## Query 1


In [ ]:
k_nearest_songs = knn_search(scaled_audio_features, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
14456,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill,"Bag, let's start the fuckin show, ah? Freak to..."
10411,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,This music does not contain words
7656,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,That shit is dope yo (Laughter) Shit was tight...
2736,1xixUOjgdqMOPZxursTms3,Anything,JAY-Z,"Uh huh yea, yeah Duro! You gotta let it bump U..."
5034,2xTft6GEZeTyWNpdX94rkf,All Eyez On Me,2Pac,"Big Syke, Newt, Hank, Beauregard, Big Sur (yea..."


In [ ]:
k_nearest_songs = knn_search_priority_queue(scaled_audio_features, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
14456,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill,"Bag, let's start the fuckin show, ah? Freak to..."
10411,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,This music does not contain words
7656,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,That shit is dope yo (Laughter) Shit was tight...
2736,1xixUOjgdqMOPZxursTms3,Anything,JAY-Z,"Uh huh yea, yeah Duro! You gotta let it bump U..."
5034,2xTft6GEZeTyWNpdX94rkf,All Eyez On Me,2Pac,"Big Syke, Newt, Hank, Beauregard, Big Sur (yea..."


## Query 2

In [ ]:
k_nearest_songs = knn_search(scaled_audio_features2, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
12225,639cc3IeyJVop1Wxm57FP3,Real Love,Drizabone,I said I would get over you Now I'm here out o...
11792,5VU2hajdO59NNjIz9p9dYD,High So High (feat. Marilyn Rylander),South Park Mexican,(Verse 1) I'm a roll '84's til them hoes start...
8167,4DmgUMnnCw27Tu5Jvqhm1n,What Do You Mean?,Skepta,The hustle is still in my genes When I fell as...
7852,496MA7l7OJ0oQtfSKHqvXH,I Gotta Clean Up,Marvin Sease,"Spoken Words I Oh, men! I almost lost her; I a..."
11316,5PM96PMKMfD1lLX2lryUsG,Oh Sheila,Ready For The World,"Like they always say, what's good for the goos..."


In [ ]:
k_nearest_songs = knn_search_priority_queue(scaled_audio_features2, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
12225,639cc3IeyJVop1Wxm57FP3,Real Love,Drizabone,I said I would get over you Now I'm here out o...
11792,5VU2hajdO59NNjIz9p9dYD,High So High (feat. Marilyn Rylander),South Park Mexican,(Verse 1) I'm a roll '84's til them hoes start...
8167,4DmgUMnnCw27Tu5Jvqhm1n,What Do You Mean?,Skepta,The hustle is still in my genes When I fell as...
7852,496MA7l7OJ0oQtfSKHqvXH,I Gotta Clean Up,Marvin Sease,"Spoken Words I Oh, men! I almost lost her; I a..."
11316,5PM96PMKMfD1lLX2lryUsG,Oh Sheila,Ready For The World,"Like they always say, what's good for the goos..."


## Query 3

In [ ]:
k_nearest_songs = knn_search(scaled_audio_features3, normalized_combined_data, k=10)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
10411,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,This music does not contain words
7656,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,That shit is dope yo (Laughter) Shit was tight...
14456,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill,"Bag, let's start the fuckin show, ah? Freak to..."
14652,74Yw8aWE60rlSAeT49h9BS,Foe Life,Mack 10,Mack 10 you know you rule hip-hop and Ice Cube...
4493,2qgudkFpaRq2YcJDWhKW44,Love and Death,Ebo Taylor,Brothers and sisters Lend me your ears Listen ...
5034,2xTft6GEZeTyWNpdX94rkf,All Eyez On Me,2Pac,"Big Syke, Newt, Hank, Beauregard, Big Sur (yea..."
6674,3p50PUgtBHM6rwUuD3RfFV,One Time 4 Your Mind,Nas,"NA One time for your mind, one time Yeah, what..."
10177,59R1Pv7IfNDHU76u6eD6yW,Time Taker,Geto Boys,Scarface: As I open my eyes unto a new day I s...
12733,6dQJI4hFolhMnZhSHxkuGc,Mama Said Knock You Out,LL Cool J,"C'mon, man And with the local DBT news, LL Coo..."
3860,2hSiACXW5Nt0yUxWUsoIpU,Mama Said Knock You Out,LL Cool J,"C'mon, man And with the local DBT news, LL Coo..."


In [ ]:
k_nearest_songs = knn_search_priority_queue(scaled_audio_features3, normalized_combined_data, k=10)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
10411,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,This music does not contain words
7656,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,That shit is dope yo (Laughter) Shit was tight...
14456,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill,"Bag, let's start the fuckin show, ah? Freak to..."
14652,74Yw8aWE60rlSAeT49h9BS,Foe Life,Mack 10,Mack 10 you know you rule hip-hop and Ice Cube...
4493,2qgudkFpaRq2YcJDWhKW44,Love and Death,Ebo Taylor,Brothers and sisters Lend me your ears Listen ...
5034,2xTft6GEZeTyWNpdX94rkf,All Eyez On Me,2Pac,"Big Syke, Newt, Hank, Beauregard, Big Sur (yea..."
6674,3p50PUgtBHM6rwUuD3RfFV,One Time 4 Your Mind,Nas,"NA One time for your mind, one time Yeah, what..."
10177,59R1Pv7IfNDHU76u6eD6yW,Time Taker,Geto Boys,Scarface: As I open my eyes unto a new day I s...
3860,2hSiACXW5Nt0yUxWUsoIpU,Mama Said Knock You Out,LL Cool J,"C'mon, man And with the local DBT news, LL Coo..."
12733,6dQJI4hFolhMnZhSHxkuGc,Mama Said Knock You Out,LL Cool J,"C'mon, man And with the local DBT news, LL Coo..."


## De la clase 

### Query 1

In [ ]:
k_nearest_songs = knn_search_simple(scaled_audio_features.to_numpy(), normalized_combined_data, k=15)
result = pd.DataFrame(k_nearest_songs)
result

,track_id,track_name,track_artist,distance,query_row
0,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill,8.117248,0
1,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,8.139057,0
2,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,8.334966,0
3,1xixUOjgdqMOPZxursTms3,Anything,JAY-Z,9.190544,0
4,2xTft6GEZeTyWNpdX94rkf,All Eyez On Me,2Pac,9.331679,0
5,5i3qlYyzV6nc4EvCrS1YVY,Manifest,Zayvin Lakin,9.434058,0
6,1dJNuQQ5Zp06oBlnwpiNzT,Mass Appeal,Gang Starr,9.639796,0
7,3lGBvPUgO7MJltUnBlOpe9,Mass Appeal,Gang Starr,9.639796,0
8,5a251qN4R86hro8UZGuO4E,The Streets - Re-Twist,WC,9.727417,0
9,5VU2hajdO59NNjIz9p9dYD,High So High (feat. Marilyn Rylander),South Park Mexican,9.730754,0


In [ ]:
k_nearest_songs = knn_search_simple_priority_queue(scaled_audio_features.to_numpy(), normalized_combined_data, k=15)
result = pd.DataFrame(k_nearest_songs)
result

,track_id,track_name,track_artist,distance,query_row
0,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill,8.117248,0
1,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,8.139057,0
2,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,8.334966,0
3,1xixUOjgdqMOPZxursTms3,Anything,JAY-Z,9.190544,0
4,2xTft6GEZeTyWNpdX94rkf,All Eyez On Me,2Pac,9.331679,0
5,5i3qlYyzV6nc4EvCrS1YVY,Manifest,Zayvin Lakin,9.434058,0
6,1dJNuQQ5Zp06oBlnwpiNzT,Mass Appeal,Gang Starr,9.639796,0
7,3lGBvPUgO7MJltUnBlOpe9,Mass Appeal,Gang Starr,9.639796,0
8,5a251qN4R86hro8UZGuO4E,The Streets - Re-Twist,WC,9.727417,0
9,5VU2hajdO59NNjIz9p9dYD,High So High (feat. Marilyn Rylander),South Park Mexican,9.730754,0


## Query 2


In [ ]:
k_nearest_songs = knn_search_simple(scaled_audio_features2.to_numpy(), normalized_combined_data, k=15)
result = pd.DataFrame(k_nearest_songs)
result

,track_id,track_name,track_artist,distance,query_row
0,639cc3IeyJVop1Wxm57FP3,Real Love,Drizabone,16.272523,0
1,5VU2hajdO59NNjIz9p9dYD,High So High (feat. Marilyn Rylander),South Park Mexican,16.292411,0
2,4DmgUMnnCw27Tu5Jvqhm1n,What Do You Mean?,Skepta,16.349711,0
3,496MA7l7OJ0oQtfSKHqvXH,I Gotta Clean Up,Marvin Sease,16.438665,0
4,5PM96PMKMfD1lLX2lryUsG,Oh Sheila,Ready For The World,16.453459,0
5,30MFmjElHOCOhfu6q48Wtv,That Girl,MJG,16.504655,0
6,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,16.570251,0
7,3ywXd57iSPA37YT0BtmP3u,When We Get By,D'Angelo,16.620315,0
8,5UeZsDGv8sYztBH8FJaPRD,Loyalty,D12,16.623186,0
9,1rylJ0VXvhOzLGNm401iaI,Keep Their Heads Ringin',Dr. Dre,16.688936,0


In [ ]:
k_nearest_songs = knn_search_simple_priority_queue(scaled_audio_features2.to_numpy(), normalized_combined_data, k=15)
result = pd.DataFrame(k_nearest_songs)
result

,track_id,track_name,track_artist,distance,query_row
0,639cc3IeyJVop1Wxm57FP3,Real Love,Drizabone,16.272523,0
1,5VU2hajdO59NNjIz9p9dYD,High So High (feat. Marilyn Rylander),South Park Mexican,16.292411,0
2,4DmgUMnnCw27Tu5Jvqhm1n,What Do You Mean?,Skepta,16.349711,0
3,496MA7l7OJ0oQtfSKHqvXH,I Gotta Clean Up,Marvin Sease,16.438665,0
4,5PM96PMKMfD1lLX2lryUsG,Oh Sheila,Ready For The World,16.453459,0
5,30MFmjElHOCOhfu6q48Wtv,That Girl,MJG,16.504655,0
6,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,16.570251,0
7,3ywXd57iSPA37YT0BtmP3u,When We Get By,D'Angelo,16.620315,0
8,5UeZsDGv8sYztBH8FJaPRD,Loyalty,D12,16.623186,0
9,1rylJ0VXvhOzLGNm401iaI,Keep Their Heads Ringin',Dr. Dre,16.688936,0


## Query 3

In [ ]:
k_nearest_songs = knn_search_simple(scaled_audio_features3.to_numpy(), normalized_combined_data, k=15)
result = pd.DataFrame(k_nearest_songs)
result

,track_id,track_name,track_artist,distance,query_row
0,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,8.975415,0
1,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,9.256692,0
2,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill,9.346621,0
3,74Yw8aWE60rlSAeT49h9BS,Foe Life,Mack 10,9.964023,0
4,2qgudkFpaRq2YcJDWhKW44,Love and Death,Ebo Taylor,9.987797,0
5,2xTft6GEZeTyWNpdX94rkf,All Eyez On Me,2Pac,10.215569,0
6,3p50PUgtBHM6rwUuD3RfFV,One Time 4 Your Mind,Nas,10.352180,0
7,59R1Pv7IfNDHU76u6eD6yW,Time Taker,Geto Boys,10.378169,0
8,2hSiACXW5Nt0yUxWUsoIpU,Mama Said Knock You Out,LL Cool J,10.428863,0
9,6dQJI4hFolhMnZhSHxkuGc,Mama Said Knock You Out,LL Cool J,10.428863,0


In [ ]:
k_nearest_songs = knn_search_simple_priority_queue(scaled_audio_features3.to_numpy(), normalized_combined_data, k=15)
result = pd.DataFrame(k_nearest_songs)
result

,track_id,track_name,track_artist,distance,query_row
0,5CwOUooch74h0XarhDfAQK,Rain Forest and Tropical Beach Sound,Nature Sounds Nature Music,8.975415,0
1,444KztZ7hPaz6fpZHYOSdD,Episode VIII,Madlib,9.256692,0
2,6zTABPrCYA8fm5ZpWrzKbz,Latin Lingo - Blackout Mix,Cypress Hill,9.346621,0
3,74Yw8aWE60rlSAeT49h9BS,Foe Life,Mack 10,9.964023,0
4,2qgudkFpaRq2YcJDWhKW44,Love and Death,Ebo Taylor,9.987797,0
5,2xTft6GEZeTyWNpdX94rkf,All Eyez On Me,2Pac,10.215569,0
6,3p50PUgtBHM6rwUuD3RfFV,One Time 4 Your Mind,Nas,10.352180,0
7,59R1Pv7IfNDHU76u6eD6yW,Time Taker,Geto Boys,10.378169,0
8,2hSiACXW5Nt0yUxWUsoIpU,Mama Said Knock You Out,LL Cool J,10.428863,0
9,6dQJI4hFolhMnZhSHxkuGc,Mama Said Knock You Out,LL Cool J,10.428863,0


## Range Search
El range search utilizamos el modelo KNN y obtenemos resultados interesantes.

In [ ]:
from sklearn.neighbors import RadiusNeighborsClassifier


def range_search(query_features=pd.DataFrame(), reference_features=pd.DataFrame(), radius=int()):
    # Solo trabajaremos con el vector característico de las canciones
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)

    # Convertimos el dataframe de la consulta a un array numpy
    query_features_numeric = query_features.to_numpy()

    # Creamos el modelo RadiusNeighborsClassifier
    knn_model = RadiusNeighborsClassifier(radius=radius, metric='euclidean')

    # Entrenamos al modelo con el vector característico de las canciones
    knn_model.fit(reference_features_numeric, reference_features['track_id'])

    # Buscamos vecinos dentro del radio para cada punto de la consulta
    neighbor_indices = knn_model.radius_neighbors(query_features_numeric, return_distance=False)

    # Extraemos información de las canciones encontradas
    song_data = []
    unique_tracks = set()

    for query_index, indices in enumerate(neighbor_indices):
        for neighbor_index in indices:
            track_id = reference_features['track_id'].iloc[neighbor_index]
            if track_id not in unique_tracks:
                unique_tracks.add(track_id)
                song_data.append({
                    'track_id': track_id,
                    'track_name': reference_features['track_name'].iloc[neighbor_index],
                    'track_artist': reference_features['track_artist'].iloc[neighbor_index],
                })

    return song_data

In [ ]:
def range_search_Simple(Q, collection, r):
    result = []

    for _, row in collection.iterrows():
        C = row.iloc[4:].to_numpy()  # Obtener el vector característico desde la quinta columna hasta el final

        # Calcular la distancia entre el objeto de consulta Q y el objeto C en la colección
        dist = distance(Q, C)

        # Verificar si la distancia es menor que el umbral r
        if dist < r:
            result.append({
                'track_id': row['track_id'],
                'track_name': row['track_name'],
                'track_artist': row['track_artist'],
                'distance': dist,
            })

    return result


## Query 1

### Modelo KNN - arbol KD

In [ ]:
radius = 11
indices_within_radius = range_search(scaled_audio_features, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result


/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RadiusNeighborsClassifier was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,05vpswcCkiXX2egk5GBo2y,Middle Of The Night,MJG
1,05xA64s3UzOOmAnh9SIIjn,Bloom,bLAck pARty
2,1arQh74CO1oQLKrO7el3FV,Devil's Pie,D'Angelo
3,1dJNuQQ5Zp06oBlnwpiNzT,Mass Appeal,Gang Starr
4,1K6oIHSzqcLpToFiwIDSI7,Memories,Cypress Hill
5,1xixUOjgdqMOPZxursTms3,Anything,JAY-Z
6,22t3RGAjsN5qaUD3GmaGHx,Played Like a Piano,King T
7,24Jk4wYlLRXI9TUMdkHoMN,Again,Wun Two
8,2CPturRUlpvirYr7VpkXCV,It Ain't Hard to Tell,Nas
9,2hSiACXW5Nt0yUxWUsoIpU,Mama Said Knock You Out,LL Cool J


### Visto en clase

In [ ]:
radius = 11
indices_within_radius = range_search_Simple(scaled_audio_features.to_numpy(), normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result


,track_id,track_name,track_artist,distance
0,05vpswcCkiXX2egk5GBo2y,Middle Of The Night,MJG,10.183579
1,05xA64s3UzOOmAnh9SIIjn,Bloom,bLAck pARty,10.463649
2,1arQh74CO1oQLKrO7el3FV,Devil's Pie,D'Angelo,10.009469
3,1dJNuQQ5Zp06oBlnwpiNzT,Mass Appeal,Gang Starr,9.639796
4,1K6oIHSzqcLpToFiwIDSI7,Memories,Cypress Hill,10.621929
5,1xixUOjgdqMOPZxursTms3,Anything,JAY-Z,9.190544
6,22t3RGAjsN5qaUD3GmaGHx,Played Like a Piano,King T,10.265777
7,24Jk4wYlLRXI9TUMdkHoMN,Again,Wun Two,10.315145
8,2CPturRUlpvirYr7VpkXCV,It Ain't Hard to Tell,Nas,9.828911
9,2hSiACXW5Nt0yUxWUsoIpU,Mama Said Knock You Out,LL Cool J,10.704407


## Query 2

### Modelo KNN - arbol KD

In [ ]:
radius = 19
indices_within_radius = range_search(scaled_audio_features2, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RadiusNeighborsClassifier was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,03GzYByXsXHcoEAxjsWv9G,Slow Motion,Color Me Badd
1,05KCB1XhTrdEASLYqbESny,Nothin' But The Cavi Hit,Mack 10
2,05VHidqx1tV6V7MsCdAIby,Lucifer,JAY-Z
3,05xA64s3UzOOmAnh9SIIjn,Bloom,bLAck pARty
4,07fqonJcksfEG2Fav9SDA2,Make Believe,Kenny Lattimore
...,...,...,...
527,7HyOh5NYicEElXbpZCuNvF,Do You Everynite,Joe Public
528,7IDKIfRltN07TdsdTVtRHh,All Over,CRUISR
529,7Je4aDuchYqv2YWMudH5ZM,Door #1,LSG
530,7JG411Bqg3I54wvm7QYwxB,Shinin',DJ DMD


### Visto en Clase

In [ ]:
radius = 19
indices_within_radius = range_search_Simple(scaled_audio_features2.to_numpy(), normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

,track_id,track_name,track_artist,distance
0,03GzYByXsXHcoEAxjsWv9G,Slow Motion,Color Me Badd,18.991706
1,05KCB1XhTrdEASLYqbESny,Nothin' But The Cavi Hit,Mack 10,17.518576
2,05VHidqx1tV6V7MsCdAIby,Lucifer,JAY-Z,18.555365
3,05xA64s3UzOOmAnh9SIIjn,Bloom,bLAck pARty,16.822184
4,07fqonJcksfEG2Fav9SDA2,Make Believe,Kenny Lattimore,17.450380
...,...,...,...,...
527,7HyOh5NYicEElXbpZCuNvF,Do You Everynite,Joe Public,18.345888
528,7IDKIfRltN07TdsdTVtRHh,All Over,CRUISR,18.135666
529,7Je4aDuchYqv2YWMudH5ZM,Door #1,LSG,17.738234
530,7JG411Bqg3I54wvm7QYwxB,Shinin',DJ DMD,17.459596


## Query 3

### Modelo KNN - arbol KD

In [ ]:
radius = 12
indices_within_radius = range_search(scaled_audio_features3, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RadiusNeighborsClassifier was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,05vpswcCkiXX2egk5GBo2y,Middle Of The Night,MJG
1,05xA64s3UzOOmAnh9SIIjn,Bloom,bLAck pARty
2,0Acc8Z2mhTiU6ceB4DEyCt,Los Angeles Times,Xzibit
3,13XHuE00ElL5thSxkaEXxK,Only God Can Judge Me,2Pac
4,1arQh74CO1oQLKrO7el3FV,Devil's Pie,D'Angelo
...,...,...,...
91,7ABJ2azouB9EwEYzL9Tljt,Booti Call,Blackstreet
92,7AitCDHecfsGYIb9nhrB5q,Magnolia's Room,Anthony Hamilton
93,7fXb8yG4gCsioqzJNZOv69,Where The Hood At,DMX
94,7IfywdSxY0okFji12beS6Y,Now I Feel Ya,Scarface


### Visto en clase

In [ ]:
radius = 12
indices_within_radius = range_search_Simple(scaled_audio_features3.to_numpy(), normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

,track_id,track_name,track_artist,distance
0,05vpswcCkiXX2egk5GBo2y,Middle Of The Night,MJG,10.848113
1,05xA64s3UzOOmAnh9SIIjn,Bloom,bLAck pARty,10.719427
2,0Acc8Z2mhTiU6ceB4DEyCt,Los Angeles Times,Xzibit,11.997982
3,13XHuE00ElL5thSxkaEXxK,Only God Can Judge Me,2Pac,11.198068
4,1arQh74CO1oQLKrO7el3FV,Devil's Pie,D'Angelo,11.700459
...,...,...,...,...
91,7ABJ2azouB9EwEYzL9Tljt,Booti Call,Blackstreet,10.903168
92,7AitCDHecfsGYIb9nhrB5q,Magnolia's Room,Anthony Hamilton,11.459694
93,7fXb8yG4gCsioqzJNZOv69,Where The Hood At,DMX,10.824240
94,7IfywdSxY0okFji12beS6Y,Now I Feel Ya,Scarface,11.151016


## Aclaraciones

La data frame tiene una cancion pero de distintas versiones y tiempo, cuenta con detalles que otros no tienen.

In [ ]:
# Filtrar el DataFrame original para obtener todas las instancias de la canción con track_id específico
all_instances = songs[songs['track_id'] == '05xbEV5M8DAQmhFKKlHaDd']

# Contar el número de instancias
num_instances = all_instances.shape[0]

# Imprimir el resultado
print(f"La canción con track_id '05xbEV5M8DAQmhFKKlHaDd' se repite {num_instances} veces.")


La canción con track_id '05xbEV5M8DAQmhFKKlHaDd' se repite 1 veces.


In [ ]:
# Identificar posibles versiones duplicadas basadas en ciertas columnas (por ejemplo, artist_name y title)
possible_duplicates = songs[songs.duplicated(['track_name', 'track_album_name'], keep=False)]

# Mostrar el DataFrame con posibles versiones duplicadas
possible_duplicates


,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.007230,0.4890,0.650,111.904,262467,en
10,00hdjyXt6MohKnCyDmhxOL,Una Vaina Loca,Fuego,Fuego Uoh uoh uoh La musiica del futuroo Yeah!...,1,01nV3KuocS1NJHTsJbPkTO,Una Vaina Loca,2011-11-02,MIX LATIN POP°,6IS6XTdbS9qJZgfjNKgpB8,...,-5.589,1,0.0361,0.11400,0.000080,0.0620,0.642,117.009,188213,es
16,00LfFm08VWeZwB0Zlm24AT,Suga Suga,Baby Bash,"So tight, so fly You got me lifted, you got me...",8,2bAkYizbM1rMbOUHpfAV9z,Tha Smokin' Nephew,2003-01-01,90s/00s Hip Hop & RnB,0Ar0Ng9DlAWZtSPBvOQgOa,...,-3.041,0,0.2680,0.68800,0.000008,0.0841,0.535,82.331,239027,en
41,01auWCuHXubb8KPZ0IXF4S,Get Get Down - Radio Edit,Merk & Kremont,"Down, down do-down, now ge-get",37,1P5jv5F4eW3TPh5k10I3Uc,Get Get Down,2015-06-05,Alex Workout,7fRlDx99wBg6i45OCvxwKY,...,-4.087,0,0.0764,0.00724,0.013400,0.4680,0.332,128.032,181875,pl
42,01Bw3xiZ5PQfn0btMWICaQ,Beautiful Life,Lost Frequencies,"Anywhere you are and you'll go, I follow Mount...",30,2WGceQjphDCN6OdXxNBH6x,Beautiful Life,2016-06-03,Tropical Beats,37i9dQZF1DX1bDvyV0IkZ3,...,-6.684,0,0.0376,0.12700,0.034700,0.1390,0.339,119.979,161464,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18382,7yCPwWs66K8Ba5lFuU2bcx,All The Small Things,blink-182,"All the small things True care, truth brings I...",21,5qt11cWjSs5Gbqj2Wyfu38,Enema Of The State,1999-01-01,keg party jukebox,4PjiT7SWnUsTTTvKOuR76f,...,-4.764,1,0.0570,0.01220,0.000000,0.5470,0.695,148.599,168000,tl
18393,7ykaUgkdQWJLsMuOymTV2A,If Not For You,George Harrison,"If not for you Babe, I couldn't even find the ...",59,4I4xtHaIFOzhZfp1NIHkY6,All Things Must Pass (Remastered),1970-11-27,I didn’t know perm stood for permanent (wave),3e6gYPyrTbaB8BWgSHCt5j,...,-6.886,1,0.0317,0.21900,0.003990,0.0820,0.867,116.119,210720,en
18400,7yqU5HODQU91zj8ahUaUX1,I Wanna Dance with Somebody (Who Loves Me),Whitney Houston,Clock strikes upon the hour And the sun begins...,2,1wvEC2yY7koRQYebhrj1ZY,Whitney,1987-06-02,80s Pop & Rock Hits and Album Tracks,0XOIK4m26aeYSD61E5nSVW,...,-8.824,1,0.0453,0.20700,0.000307,0.0888,0.867,118.814,291293,en
18416,7zdoYASH8iYGR1CQV3m8dY,SOS,McClenney,"Hey, hey, oh, oh Hey, hey, oh, oh Hey, hey, oh...",5,7CHix0EuA9xs2RY2sFi5Pv,SOS,2019-02-19,I Love My Neo-Soul,37i9dQZF1DX44dZ4p5QLf4,...,-13.628,0,0.0390,0.57200,0.003940,0.1220,0.555,72.950,187242,en


In [ ]:
# Seleccionar la canción específica (cambiar 'track_name' y 'track_album_name' según tus necesidades)
cancion_seleccionada = songs[(songs['track_name'] == 'Nobody (feat. Athena Cage)')]

# Contar duplicados para la canción seleccionada
duplicados_cancion_seleccionada = cancion_seleccionada.shape[0]

# Imprimir el resultado
print(f"La canción seleccionada se repite {duplicados_cancion_seleccionada} veces.")

cancion_seleccionada


La canción seleccionada se repite 1 veces.


,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
11252,4SF1747p541umnykBp352Q,Nobody (feat. Athena Cage),Keith Sweat,I wanna tease you I wanna please you I wanna s...,62,0BzXvdpUKDEk612hLc6rZV,Keith Sweat,1996,"Swingbeat (old skool), New Jack Swing, R&B, Hi...",3krpccUV68nBGAQbvHEZDC,...,-9.449,0,0.0318,0.484,0.0,0.0482,0.388,117.907,264773,en
